In [76]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
import seaborn as sns
from sklearn.calibration import cross_val_predict
from sklearn.model_selection import KFold, cross_val_score
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import plotly.express as px
import pandas as pd
import psutil
import os
import distro
import platform
import sys
import tracemalloc
import numpy as np
import torch
import platform
import subprocess
import pyRAPL
import plotly.graph_objects as go
import time

In [41]:
# Load dataset
path = 'lcs.csv'

df = pd.read_csv(path)

df

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,F,74,1,1,1,1,1,1,1,1,1,2,1,1,1,NO
1153,F,75,1,1,1,1,1,1,1,1,1,2,1,1,1,NO
1154,F,76,1,1,1,1,1,1,1,1,1,2,1,1,1,NO
1155,F,77,1,1,1,1,1,1,1,1,1,2,1,1,1,NO


In [42]:
df.shape

(1157, 16)

In [43]:
df.columns

Index(['GENDER', 'AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY',
       'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
       'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
       'SWALLOWING DIFFICULTY', 'CHEST PAIN', 'LUNG_CANCER'],
      dtype='object')

In [44]:
print(df.dtypes)

GENDER                   object
AGE                       int64
SMOKING                   int64
YELLOW_FINGERS            int64
ANXIETY                   int64
PEER_PRESSURE             int64
CHRONIC DISEASE           int64
FATIGUE                   int64
ALLERGY                   int64
WHEEZING                  int64
ALCOHOL CONSUMING         int64
COUGHING                  int64
SHORTNESS OF BREATH       int64
SWALLOWING DIFFICULTY     int64
CHEST PAIN                int64
LUNG_CANCER              object
dtype: object


In [45]:
import plotly.express as px

# Membuat DataFrame tdf yang berisi counts dari variabel 'Level'
tdf = df['LUNG_CANCER'].value_counts().reset_index()
tdf.columns = ['LUNG_CANCER', 'count']  # Memberi nama kolom agar lebih mudah dipahami

# Menambahkan kolom persentase
total_count = tdf['count'].sum()
tdf['percentage'] = (tdf['count'] / total_count) * 100

# Membuat diagram batang interaktif menggunakan Plotly
fig = px.bar(tdf, x='LUNG_CANCER', y='count',
             title='Lung Cancer Distribution',
             labels={'LUNG_CANCER': 'Lung Cancer Category', 'count': 'Count'},
             color='LUNG_CANCER',  # Memberikan warna berbeda untuk setiap kategori
             color_discrete_sequence=px.colors.qualitative.Set1,  # Menentukan palet warna
             text=tdf.apply(lambda row: f"{int(row['count'])} ({row['percentage']:.1f}%)", axis=1))  # Menambahkan count dan persentase

# Menampilkan chart
fig.update_traces(textposition='outside',  # Menempatkan teks di luar batang
                  texttemplate='%{text}')  # Menampilkan nilai count dan persentase

# Menyesuaikan margin agar teks tidak tertutup
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),  # Memberikan ruang pada sisi kiri, kanan, atas, dan bawah
)

fig.show()


In [46]:
print(df.isnull().sum())

GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL CONSUMING        0
COUGHING                 0
SHORTNESS OF BREATH      0
SWALLOWING DIFFICULTY    0
CHEST PAIN               0
LUNG_CANCER              0
dtype: int64


Tahap Preprocessing (Encoding tipe data)

In [47]:
# Mengubah kolom 'GENDER' menjadi numerik (F = 0, M = 1)
df['GENDER'] = df['GENDER'].map({'F': 0, 'M': 1})

# Mengubah kolom 'LUNG_CANCER' menjadi numerik (NO = 0, YES = 1)
df['LUNG_CANCER'] = df['LUNG_CANCER'].map({'NO': 0, 'YES': 1})

# Cek hasilnya
print(df[['GENDER', 'LUNG_CANCER']].head())

   GENDER  LUNG_CANCER
0       1            1
1       1            1
2       0            0
3       1            0
4       0            0


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157 entries, 0 to 1156
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   GENDER                 1157 non-null   int64
 1   AGE                    1157 non-null   int64
 2   SMOKING                1157 non-null   int64
 3   YELLOW_FINGERS         1157 non-null   int64
 4   ANXIETY                1157 non-null   int64
 5   PEER_PRESSURE          1157 non-null   int64
 6   CHRONIC DISEASE        1157 non-null   int64
 7   FATIGUE                1157 non-null   int64
 8   ALLERGY                1157 non-null   int64
 9   WHEEZING               1157 non-null   int64
 10  ALCOHOL CONSUMING      1157 non-null   int64
 11  COUGHING               1157 non-null   int64
 12  SHORTNESS OF BREATH    1157 non-null   int64
 13  SWALLOWING DIFFICULTY  1157 non-null   int64
 14  CHEST PAIN             1157 non-null   int64
 15  LUNG_CANCER            1157 non-null  

In [49]:
# Pisahkan data
X = df.drop(columns=['LUNG_CANCER'])  # Semua kolom kecuali 'LUNG_CANCER'
y = df['LUNG_CANCER']

In [68]:
X.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN
0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2
1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2
2,0,59,1,1,1,2,1,2,1,2,1,2,2,1,2
3,1,63,2,2,2,1,1,1,1,1,2,1,1,2,2
4,0,63,1,2,1,1,1,1,1,2,1,2,2,1,1


In [67]:
y.head()

0    1
1    1
2    0
3    0
4    0
Name: LUNG_CANCER, dtype: int64

In [51]:
# data hasil preprocessing
df

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,0,59,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,1,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,0,63,1,2,1,1,1,1,1,2,1,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,0,74,1,1,1,1,1,1,1,1,1,2,1,1,1,0
1153,0,75,1,1,1,1,1,1,1,1,1,2,1,1,1,0
1154,0,76,1,1,1,1,1,1,1,1,1,2,1,1,1,0
1155,0,77,1,1,1,1,1,1,1,1,1,2,1,1,1,0


kFold 10 Split

In [ ]:
# Angka 42 sering dipakai sebagai default seed "tradisional" di dunia pemrograman dan machine learning.
# Ini adalah referensi dari buku "The Hitchhiker's Guide to the Galaxy" di mana 42 adalah "the answer to the ultimate question of life, the universe, and everything".
kf = KFold(n_splits=10, shuffle=True, random_state=42) 

In [53]:
def evaluate_model_kfold(model, info):
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    aucs = []

    rows = []

    print(f"==== {info} ====")

    for fold, (train_index, val_index) in enumerate(kf.split(X), start=1):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        y_proba = model.predict_proba(X_val)[:, 1]  # Probabilitas kelas positif (1)

        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred, average='binary')
        rec = recall_score(y_val, y_pred, average='binary')
        f1 = f1_score(y_val, y_pred, average='binary')
        auc = roc_auc_score(y_val, y_proba)

        accuracies.append(acc)
        precisions.append(prec)
        recalls.append(rec)
        f1_scores.append(f1)
        aucs.append(auc)

        rows.append({
            "Fold": fold,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1,
            "AUC": auc
        })

        print(f"Fold {fold}: Accuracy={acc:.2f}, Precision={prec:.2f}, Recall={rec:.2f}, F1-Score={f1:.2f}, AUC={auc:.2f}")

    rows.append({
        "Fold": "Mean",
        "Accuracy": np.mean(accuracies),
        "Precision": np.mean(precisions),
        "Recall": np.mean(recalls),
        "F1-Score": np.mean(f1_scores),
        "AUC": np.mean(aucs)
    })

    print(f"\n--- Mean Metrics of {info} ---")
    print(f"Mean Accuracy:  {np.mean(accuracies):.2f}")
    print(f"Mean Precision: {np.mean(precisions):.2f}")
    print(f"Mean Recall:    {np.mean(recalls):.2f}")
    print(f"Mean F1-Score:  {np.mean(f1_scores):.2f}")
    print(f"Mean AUC:       {np.mean(aucs):.2f}")

    # Simpan ke folder "result/"
    os.makedirs("result", exist_ok=True)
    filename = f"result/{info.lower().replace(' ', '_')}_evaluation.csv"
    pd.DataFrame(rows).to_csv(filename, index=False)
    print(f"\n📁 Hasil evaluasi disimpan ke: {filename}")

In [54]:
dtree = DecisionTreeClassifier(random_state=42)

evaluate_model_kfold(dtree, "Decision Tree")

==== Decision Tree ====
Fold 1: Accuracy=0.95, Precision=0.98, Recall=0.91, F1-Score=0.94, AUC=0.93
Fold 2: Accuracy=0.92, Precision=0.98, Recall=0.85, F1-Score=0.91, AUC=0.89
Fold 3: Accuracy=0.97, Precision=0.96, Recall=0.96, F1-Score=0.96, AUC=0.96
Fold 4: Accuracy=0.91, Precision=0.96, Recall=0.85, F1-Score=0.90, AUC=0.89
Fold 5: Accuracy=0.93, Precision=0.98, Recall=0.88, F1-Score=0.93, AUC=0.91
Fold 6: Accuracy=0.95, Precision=0.98, Recall=0.90, F1-Score=0.94, AUC=0.92
Fold 7: Accuracy=0.93, Precision=0.98, Recall=0.86, F1-Score=0.91, AUC=0.90
Fold 8: Accuracy=0.94, Precision=0.93, Recall=0.91, F1-Score=0.92, AUC=0.92
Fold 9: Accuracy=0.95, Precision=1.00, Recall=0.86, F1-Score=0.93, AUC=0.90
Fold 10: Accuracy=0.89, Precision=0.96, Recall=0.80, F1-Score=0.87, AUC=0.85

--- Mean Metrics of Decision Tree ---
Mean Accuracy:  0.93
Mean Precision: 0.97
Mean Recall:    0.88
Mean F1-Score:  0.92
Mean AUC:       0.91

📁 Hasil evaluasi disimpan ke: result/decision_tree_evaluation.csv


In [55]:
rf = RandomForestClassifier(random_state=42)

evaluate_model_kfold(rf, "Random Forest")

==== Random Forest ====


Fold 1: Accuracy=0.97, Precision=0.98, Recall=0.94, F1-Score=0.96, AUC=0.98
Fold 2: Accuracy=0.94, Precision=0.98, Recall=0.89, F1-Score=0.93, AUC=0.92
Fold 3: Accuracy=0.96, Precision=0.94, Recall=0.96, F1-Score=0.95, AUC=0.96
Fold 4: Accuracy=0.91, Precision=0.94, Recall=0.87, F1-Score=0.90, AUC=0.92
Fold 5: Accuracy=0.92, Precision=0.98, Recall=0.86, F1-Score=0.92, AUC=0.93
Fold 6: Accuracy=0.95, Precision=0.98, Recall=0.90, F1-Score=0.94, AUC=0.95
Fold 7: Accuracy=0.91, Precision=0.95, Recall=0.84, F1-Score=0.89, AUC=0.93
Fold 8: Accuracy=0.93, Precision=0.91, Recall=0.91, F1-Score=0.91, AUC=0.94
Fold 9: Accuracy=0.95, Precision=0.97, Recall=0.89, F1-Score=0.93, AUC=0.93
Fold 10: Accuracy=0.90, Precision=0.96, Recall=0.82, F1-Score=0.88, AUC=0.91

--- Mean Metrics of Random Forest ---
Mean Accuracy:  0.93
Mean Precision: 0.96
Mean Recall:    0.89
Mean F1-Score:  0.92
Mean AUC:       0.94

📁 Hasil evaluasi disimpan ke: result/random_forest_evaluation.csv


In [57]:
def measure_execute_time(model):
    execution_times = []

    for _, (train_index, _) in enumerate(kf.split(X), start=1):
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]

        start_time = time.time()
        model.fit(X_train, y_train)
        end_time = time.time()

        elapsed_time = end_time - start_time
        execution_times.append(elapsed_time)

    return execution_times


In [58]:
# mengukur waktu eksekusi DT
execution_times_dt = measure_execute_time(dtree)

# Ambil nilai terkecil dan terbesar
min_time_dt = min(execution_times_dt)
max_time_dt = max(execution_times_dt)

# Tampilkan hasil
print(f"Execution Time (Min): {min_time_dt:.4f} seconds")
print(f"Execution Time (Max): {max_time_dt:.4f} seconds")

Execution Time (Min): 0.0120 seconds
Execution Time (Max): 0.0189 seconds


In [59]:
# mengukur waktu eksekusi RF
execution_times_rf = measure_execute_time(rf)

# Ambil nilai terkecil dan terbesar
min_time_rf = min(execution_times_rf)
max_time_rf = max(execution_times_rf)

# Tampilkan hasil
print(f"Execution Time (Min): {min_time_rf:.4f} seconds")
print(f"Execution Time (Max): {max_time_rf:.4f} seconds")

Execution Time (Min): 0.6914 seconds
Execution Time (Max): 0.9549 seconds


In [60]:
# Fungsi untuk mendapatkan penggunaan RAM awal dan akhir dari tracemalloc
def process_memory():
    snapshot, _ = tracemalloc.get_traced_memory()
    return snapshot / (1024 * 1024)  # Konversi ke MB

def measure_memory_usage(model):
    memory_usages = []

    for train_index, _ in kf.split(X):
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]

        tracemalloc.start()
        initial_memory = process_memory()
        model.fit(X_train, y_train)
        final_memory = process_memory()
        tracemalloc.stop()

        memory_usage = final_memory - initial_memory
        memory_usages.append(memory_usage)

    return memory_usages

In [61]:
# Memory usage DT
memory_usages_dt = measure_memory_usage(dtree)

# Ambil nilai terkecil dan terbesar
min_memory_dt, max_memory_dt = min(memory_usages_dt), max(memory_usages_dt)

# Tampilkan hasil
print(f"Memory Usage (Min): {min_memory_dt:.4f} MB")
print(f"Memory Usage (Max): {max_memory_dt:.4f} MB")

Memory Usage (Min): 0.0025 MB
Memory Usage (Max): 0.0032 MB


In [62]:
# Memory usage RF
memory_usages_rf = measure_memory_usage(rf)

# Ambil nilai terkecil dan terbesar
min_memory_rf, max_memory_rf = min(memory_usages_rf), max(memory_usages_rf)

# Tampilkan hasil
print(f"Memory Usage (Min): {min_memory_rf:.4f} MB")
print(f"Memory Usage (Max): {max_memory_rf:.4f} MB")

Memory Usage (Min): 0.0672 MB
Memory Usage (Max): 0.0704 MB


In [63]:
# gunakan: sudo chmod -R a+r /sys/class/powercap/intel-rapl diterminal, lalu restart kernel (opsional)
# untuk handle: PermissionError: [Errno 13] Permission denied: '/sys/class/powercap/intel-rapl/intel-rapl:0/energy_uj'
def measure_cpu_energy_usage(label, model):
    pyRAPL.setup()

    cpu_energies = []

    for fold, (train_index, _) in enumerate(kf.split(X), start=1):
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]

        meter = pyRAPL.Measurement(label)
        
        with meter:
            model.fit(X_train, y_train)
        
        result = meter.result
        cpu_energy_joule = result.pkg[0] * 1e-6 #convert dari microjoule ke joule
        cpu_energies.append(cpu_energy_joule)

    return min(cpu_energies), max(cpu_energies)

In [64]:
# Measure CPU energy DT
min_cpu_dt, max_cpu_dt = measure_cpu_energy_usage('dtree', dtree)

# Tampilkan hasil
print(f"CPU Energy (Min): {min_cpu_dt:.4f} J")
print(f"CPU Energy (Max): {max_cpu_dt:.4f} J")

Label : dtree
Begin : Tue May 27 16:50:07 2025
Duration : 19536.7540 us
-------------------------------
PKG :
	socket 0 :  114685.0000 uJ
-------------------------------
DRAM :
	socket 0 :  38269.0000 uJ
-------------------------------
Label : dtree
Begin : Tue May 27 16:50:07 2025
Duration : 14416.0890 us
-------------------------------
PKG :
	socket 0 :  82764.0000 uJ
-------------------------------
DRAM :
	socket 0 :  27649.0000 uJ
-------------------------------
Label : dtree
Begin : Tue May 27 16:50:07 2025
Duration : 15002.6400 us
-------------------------------
PKG :
	socket 0 :  70129.0000 uJ
-------------------------------
DRAM :
	socket 0 :  20081.0000 uJ
-------------------------------
Label : dtree
Begin : Tue May 27 16:50:07 2025
Duration : 15261.5300 us
-------------------------------
PKG :
	socket 0 :  100403.0000 uJ
-------------------------------
DRAM :
	socket 0 :  30212.0000 uJ
-------------------------------
Label : dtree
Begin : Tue May 27 16:50:07 2025
Duration : 

In [65]:
# Measure CPU energy RF
min_cpu_rf, max_cpu_rf = measure_cpu_energy_usage('random_forest', rf)

# Tampilkan hasil
print(f"CPU Energy (Min): {min_cpu_rf:.4f} J")
print(f"CPU Energy (Max): {max_cpu_rf:.4f} J")

Label : random_forest
Begin : Tue May 27 16:50:07 2025
Duration : 836790.0610 us
-------------------------------
PKG :
	socket 0 :  3995657.0000 uJ
-------------------------------
DRAM :
	socket 0 :  1483151.0000 uJ
-------------------------------
Label : random_forest
Begin : Tue May 27 16:50:08 2025
Duration : 785201.1850 us
-------------------------------
PKG :
	socket 0 :  2999931.0000 uJ
-------------------------------
DRAM :
	socket 0 :  1237851.0000 uJ
-------------------------------
Label : random_forest
Begin : Tue May 27 16:50:09 2025
Duration : 746273.8750 us
-------------------------------
PKG :
	socket 0 :  2976250.0000 uJ
-------------------------------
DRAM :
	socket 0 :  1193478.0000 uJ
-------------------------------
Label : random_forest
Begin : Tue May 27 16:50:10 2025
Duration : 722380.8880 us
-------------------------------
PKG :
	socket 0 :  2525384.0000 uJ
-------------------------------
DRAM :
	socket 0 :  1076474.0000 uJ
-------------------------------
Label : 

In [82]:
#Informasi Perangkat yang saya gunakan
# OS
print("=== Sistem Operasi (OS) Info ===")
print(f"OS Name         : {"Linux" if os.name == "posix" else "Unknown"}")
print(f"Distro Name     : {distro.name()}")
print(f"Version         : {distro.version()}")
print(f"Details         : {distro.lsb_release_info()}")
print(f"Kernel Version  : {platform.release()}")
print(f"Python Path     : {sys.executable}")
print(f"Python Version  : {platform.python_version()}")
print()

# Informasi CPU
cpu_model = subprocess.run(["cat", "/proc/cpuinfo"], capture_output=True, text=True).stdout
cpu_name = [line for line in cpu_model.split("\n") if "model name" in line]
cpu_name = cpu_name[0].split(":")[1].strip() if cpu_name else "Unknown"

cpu_architecture = platform.machine()
cpu_cores = psutil.cpu_count(logical=False)
cpu_threads = psutil.cpu_count(logical=True)
cpu_freq = psutil.cpu_freq().max if psutil.cpu_freq() else "Unknown"

# Informasi Cache CPU
cache_info = subprocess.run(["lscpu"], capture_output=True, text=True).stdout
l1_instruction_cache, l1_data_cache, l2_cache, l3_cache = "Unknown", "Unknown", "Unknown", "Unknown"

for line in cache_info.split("\n"):
    if "L1d cache" in line:
        l1_data_cache = line.split(":")[1].strip()
    elif "L1i cache" in line:
        l1_instruction_cache = line.split(":")[1].strip()
    elif "L2 cache" in line:
        l2_cache = line.split(":")[1].strip()
    elif "L3 cache" in line:
        l3_cache = line.split(":")[1].strip()

# Informasi RAM
ram = psutil.virtual_memory()
total_ram = round(ram.total / (1024**3), 2)  # Konversi ke GB

# Informasi Disk
disk = psutil.disk_usage('/')
total_disk = round(disk.total / (1024**3), 2)

# Informasi GPU (jika tersedia)
gpu_name = "None"
gpu_memory = "N/A"
gpu_info = "No GPU detected."

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    total_memory = torch.cuda.get_device_properties(0).total_memory
    gpu_memory = f"{round(total_memory / (1024**3), 2)} GB"

    # Gunakan nvidia-smi untuk detail tambahan
    nvidia_info = subprocess.run(["nvidia-smi", "--query-gpu=name,memory.total", "--format=csv"], 
                                 capture_output=True, text=True).stdout
    gpu_info = nvidia_info if nvidia_info else gpu_name

# Output
print("=== System Information ===")
print(f"CPU Model        : {cpu_name}")
print(f"CPU Architecture : {cpu_architecture}")
print(f"CPU Cores        : {cpu_cores} (Physical), {cpu_threads} (Logical)")
print(f"CPU Max Frequency: {cpu_freq} MHz")
print(f"Total RAM        : {total_ram} GB")
print(f"Total Disk       : {total_disk} GB")
print(f"GPU Model        : {gpu_name} ({gpu_memory})")

print("\n=== CPU Cache Information ===")
print(f"L1 Instruction Cache: {l1_instruction_cache}")
print(f"L1 Data Cache      : {l1_data_cache}")
print(f"L2 Cache           : {l2_cache}")
print(f"L3 Cache           : {l3_cache}")

if torch.cuda.is_available():
    print("\n=== GPU Details ===")
    print(gpu_info)
else:
    print("\nNo GPU detected.")

=== Sistem Operasi (OS) Info ===
OS Name         : Linux
Distro Name     : Ubuntu
Version         : 24.04
Details         : {'distributor_id': 'Ubuntu', 'description': 'Ubuntu 24.04.2 LTS', 'release': '24.04', 'codename': 'noble'}
Kernel Version  : 6.11.0-17-generic
Python Path     : /home/yashlan/Documents/GitHub/lung-cancer-prediction/myvenv/bin/python
Python Version  : 3.12.3

=== System Information ===
CPU Model        : Intel(R) Core(TM) i7-10610U CPU @ 1.80GHz
CPU Architecture : x86_64
CPU Cores        : 4 (Physical), 8 (Logical)
CPU Max Frequency: 4900.0 MHz
Total RAM        : 15.27 GB
Total Disk       : 63.0 GB
GPU Model        : None (N/A)

=== CPU Cache Information ===
L1 Instruction Cache: 128 KiB (4 instances)
L1 Data Cache      : 128 KiB (4 instances)
L2 Cache           : 1 MiB (4 instances)
L3 Cache           : 8 MiB (1 instance)

No GPU detected.
